# Exctract demographic info and cognitive metrics from the HCP behavioral data

In [40]:
import os, sys, time, h5py, zipfile
import numpy as np
import pandas as pd
from statistics import median
import urllib
import csv
import requests

Loading the HCP spreadsheet and select the subset of columns we are interesting on working with:
- Subject ID
- Age
- Gender
- Mini Mental State Examination (MMSE) total score
- Penn matrix to measure fluid intelligence (number of correct answers)
- Speed of processing score

In [41]:
hcp_behav = pd.read_csv('/home/anna/Scrivania/NeuroHackademy2019/unrestricted_annatruzzi91_8_5_2019_13_21_16.csv')
hcp_behav.keys()
behav_subset = hcp_behav[['Subject','Age','Gender','MMSE_Score','PMAT24_A_CR','ProcSpeed_AgeAdj']]

Ages are expressed in intervals (the actual ace is a Tier 1 restricted information).
Therefore we need to extract the min age for each participants in order to create 4 groups (one per age interval)

In [42]:
min_age_vector=[]
max_age_vector=[]
for item in behav_subset['Age']:
    if '+' in item:
        min_age_vector.append(int(item[0:2]))
        max_age_vector.append(90)
    else:
        min_age_vector.append(int(item[0:2]))
        max_age_vector.append(int(item[-2:]))

min_age_vector=np.asarray(min_age_vector)
max_age_vector=np.asarray(max_age_vector)

Inspect variables of interest (i.e. range, count occurrences, median)

In [43]:
## Age
ageunique, agecounts = np.unique(min_age_vector, return_counts=True)
print('Age')
print(dict(zip(ageunique, agecounts)))
agelist = ageunique

## MMSE 
unique, counts = np.unique(np.array(hcp_behav['MMSE_Score']), return_counts=True)
print('MMSE')
print(dict(zip(unique, counts)))

## Pemm matrices
print('Pemm')
print('min Pemm score is '+str(min(hcp_behav['PMAT24_A_CR'])))
print('max Pemm score is '+str(max(hcp_behav['PMAT24_A_CR'])))
print('median Pemm score is '+str(median(hcp_behav['PMAT24_A_CR'])))

## Processing speed
print('Processing speed')
print('min Processing speed score is '+str(min(hcp_behav['ProcSpeed_AgeAdj'])))
print('max Processing speed score is '+str(max(hcp_behav['ProcSpeed_AgeAdj'])))
print('median Processing speed score is '+str(median(hcp_behav['ProcSpeed_AgeAdj'])))

Age
{22: 247, 26: 527, 31: 418, 36: 14}
MMSE
{23: 1, 24: 1, 26: 31, 27: 84, 28: 197, 29: 440, 30: 452}
Pemm
min Pemm score is 4.0
max Pemm score is 24.0
median Pemm score is 10.0
Processing speed
min Processing speed score is 45.31
max Processing speed score is 149.3
median Processing speed score is 103.13


Add columns to the dataframe in which to define the category each participant is in for each variable of interest:
- Age --> 0 (min age = 22),1 (min age = 26),2 (min age = 31),3 (min age = 36)
- MMSE --> o (lower than median), 1 (higher than median)
- Pemm matrices --> o (lower than median), 1 (higher than median)
- Processing speed --> o (lower than median), 1 (higher than median)

In [44]:
## Age
age_category = np.zeros(shape=len(behav_subset['Age']))
behav_subset['min_age'] = pd.Series(min_age_vector)
behav_subset['age_category'] = pd.Series(age_category)
behav_subset.loc[behav_subset['min_age'] == agelist[0],['age_category']] = 0
behav_subset.loc[behav_subset['min_age'] == agelist[1],['age_category']] = 1
behav_subset.loc[behav_subset['min_age'] == agelist[2],['age_category']] = 2
behav_subset.loc[behav_subset['min_age'] == agelist[3],['age_category']] = 3

## MMSE
MMSE_level = np.zeros(shape=len(behav_subset['MMSE_Score']))
behav_subset['MMSE_Level'] = pd.Series(MMSE_level)
behav_subset.loc[behav_subset['MMSE_Score'] <= median(behav_subset['MMSE_Score']),['MMSE_Level']] = 0
behav_subset.loc[behav_subset['MMSE_Score'] > median(behav_subset['MMSE_Score']),['MMSE_Level']] = 1

## Penn matrices
Penn_level = np.zeros(shape=len(behav_subset['PMAT24_A_CR']))
behav_subset['Penn_Level'] = pd.Series(Penn_level)
behav_subset.loc[behav_subset['PMAT24_A_CR'] <= median(behav_subset['PMAT24_A_CR']),['Penn_Level']] = 0
behav_subset.loc[behav_subset['PMAT24_A_CR'] > median(behav_subset['PMAT24_A_CR']),['Penn_Level']] = 1

## Processing speed
ProcSpeed_level = np.zeros(shape=len(behav_subset['ProcSpeed_AgeAdj']))
behav_subset['ProcSpeed_Level'] = pd.Series(ProcSpeed_level)
behav_subset.loc[behav_subset['ProcSpeed_AgeAdj'] <= median(behav_subset['ProcSpeed_AgeAdj']),['ProcSpeed_Level']] = 0
behav_subset.loc[behav_subset['ProcSpeed_AgeAdj'] > median(behav_subset['ProcSpeed_AgeAdj']),['ProcSpeed_Level']] = 1

behav_subset



/home/anna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/anna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/anna/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

Subject    Age Gender  MMSE_Score  PMAT24_A_CR  ProcSpeed_AgeAdj  \
0      100004  22-25      M          29         19.0             69.88   
1      100206  26-30      M          30         20.0            131.89   
2      100307  26-30      F          29         17.0            112.36   
3      100408  31-35      M          30          7.0             96.00   
4      100610  26-30      M          30         23.0             96.12   
5      101006  31-35      F          28         11.0             71.99   
6      101107  22-25      M          29         14.0             97.87   
7      101208  31-35      F          30          8.0             96.97   
8      101309  26-30      M          29         15.0             87.33   
9      101410  26-30      M          29         19.0             91.83   
10     101612  26-30      F          28         19.0            120.69   
11     101915  31-35      F          29         21.0             90.69   
12     102008  22-25      M          30         20.0            130.65   
13     102109  26-30      M          29         21.0             71.10   
14     102311  26-30      F          30         20.0             68.81   
15     102513  26-30      M          29         18.0             46.07   
16     102614  22-25      M          29         18.0            143.56   
17     102715  26-30      M          29         22.0            118.18   
18     102816  26-30      F          29         17.0             93.01   
19     103010  22-25      M          28         19.0             87.33   
20     103111  26-30      M          26         21.0             81.57   
21     103212  31-35      M          26         15.0             86.51   
22     103414  22-25      F          29         17.0             94.90   
23     103515  26-30      F          27          8.0             82.95   
24     103818  31-35      F          28         16.0             95.98   
25     104012  26-30      F          27         10.0             92.66   
26     104113  26-30      M          27         11.0             96.12   
27     104416  31-35      F          30         23.0            133.66   
28     104820    36+      F          27         15.0            111.48   
29     105014  26-30      F          30         21.0             67.35   
...       ...    ...    ...         ...          ...               ...   
1176   953764  31-35      F          30          9.0            144.44   
1177   955465  26-30      M          29          8.0            110.48   
1178   957974  26-30      F          30         16.0             97.67   
1179   958976  26-30      M          29         14.0             92.04   
1180   959069  31-35      M          30         24.0             83.74   
1181   959574  26-30      F          30         16.0            116.81   
1182   962058  26-30      M          28         10.0            106.90   
1183   965367  31-35      M          27         15.0            131.33   
1184   965771  31-35      M          27         10.0            101.10   
1185   966975  22-25      M          28         14.0            112.44   
1186   969476  31-35      M          29         19.0            129.86   
1187   970764  22-25      F          30         16.0            108.74   
1188   971160  26-30      M          30         17.0            112.44   
1189   972566  31-35      F          27         17.0            144.44   
1190   973770  22-25      M          28         13.0            104.97   
1191   978578  31-35      F          30         20.0             91.25   
1192   979984  31-35      F          30         14.0             68.15   
1193   983773  26-30      M          30         24.0            103.49   
1194   984472  26-30      F          30         22.0            109.58   
1195   987074  22-25      M          30         23.0            134.24   
1196   987983  26-30      F          30         22.0             91.14   
1197   989987  31-35      M          29         20.0             90.05   
1198   990

In [46]:
behav_subset.to_csv('/home/anna/Scrivania/Neurohackademy2019/gan_dl_BehavioralData.csv',sep=',')

NotADirectoryError: [Errno 20] Not a directory: '/home/anna/Scrivania/Neurohackademy2019/gan_dl_BehavioralData.csv'